**Name:- Pranjal Godse------------ Batch:- 6**

#Problem 1:- Topic Modeling Research Dashboard


• Take a dataset (20 Newsgroups or research articles).
• Train both LDA and NMF models.
• Visualize topics using pyLDAvis in a dashboard.
• Compare models with coherence & perplexity scores.
• Bonus: Try BERTopic (transformer-based topic modeling).
• List item




In [10]:
import warnings
warnings.filterwarnings("ignore")


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [19]:
!pip install pyLDAvis gensim scikit-learn bertopic umap-learn


In [18]:
import numpy as np
import pandas as pd
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.metrics import pairwise_distances

import gensim
from gensim import corpora
from gensim.models import CoherenceModel
from bertopic import BERTopic


In [17]:
dataset = fetch_20newsgroups(remove=('headers', 'footers', 'quotes'))

documents = dataset.data
print("Total Documents:", len(documents))


Total Documents: 11314


In [16]:
from gensim.utils import simple_preprocess

processed_docs = [simple_preprocess(doc) for doc in documents]

dictionary = corpora.Dictionary(processed_docs)
corpus = [dictionary.doc2bow(text) for text in processed_docs]


In [15]:
lda_model = gensim.models.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=10,
    random_state=42,
    passes=10
)


In [14]:
lda_model.print_topics()


[(0,
  '0.021*"cx" + 0.017*"c_" + 0.014*"hz" + 0.011*"qs" + 0.010*"lk" + 0.010*"ck" + 0.009*"ww" + 0.007*"st" + 0.007*"chz" + 0.006*"zd"'),
 (1,
  '0.039*"the" + 0.019*"to" + 0.016*"of" + 0.016*"and" + 0.014*"is" + 0.014*"in" + 0.012*"file" + 0.010*"for" + 0.008*"edu" + 0.007*"from"'),
 (2,
  '0.676*"ax" + 0.050*"max" + 0.007*"pl" + 0.006*"ei" + 0.005*"di" + 0.005*"tm" + 0.004*"bhj" + 0.004*"giz" + 0.004*"wm" + 0.004*"ey"'),
 (3,
  '0.056*"the" + 0.030*"to" + 0.023*"and" + 0.019*"is" + 0.018*"of" + 0.016*"for" + 0.016*"it" + 0.014*"in" + 0.011*"with" + 0.011*"you"'),
 (4,
  '0.007*"tyre" + 0.004*"comic" + 0.004*"navy" + 0.004*"da" + 0.003*"baerga" + 0.003*"reproduction" + 0.003*"czech" + 0.003*"udel" + 0.003*"seminar" + 0.002*"mozumder"'),
 (5,
  '0.025*"the" + 0.012*"team" + 0.011*"game" + 0.011*"he" + 0.010*"in" + 0.009*"at" + 0.008*"year" + 0.008*"games" + 0.007*"hockey" + 0.006*"season"'),
 (6,
  '0.012*"dos" + 0.008*"cds" + 0.007*"cd" + 0.004*"mpc" + 0.003*"dance" + 0.002*"nail" +

In [13]:
coherence_lda = CoherenceModel(
    model=lda_model,
    texts=processed_docs,
    dictionary=dictionary,
    coherence='c_v'
)

print("LDA Coherence Score:", coherence_lda.get_coherence())


LDA Coherence Score: 0.6304502158042121


In [12]:
print("LDA Perplexity:", lda_model.log_perplexity(corpus))


LDA Perplexity: -7.627248357318394


In [11]:
pyLDAvis.enable_notebook()

lda_vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
lda_vis


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.307696 -0.022615       1        1  55.149841
3      0.319225 -0.029984       2        1  25.386780
1      0.229175 -0.008115       3        1   5.952883
2     -0.187834 -0.392688       4        1   4.303736
8      0.061847  0.068409       5        1   3.156062
5      0.068476  0.063188       6        1   3.077849
7     -0.196003  0.028040       7        1   1.819484
0     -0.221798  0.079603       8        1   0.930566
4     -0.191440  0.107966       9        1   0.123230
6     -0.189344  0.106195      10        1   0.099568, topic_info=              Term           Freq          Total Category  logprob  loglift
5474            ax   59876.000000   59876.000000  Default  30.0000  30.0000
51             the  108870.000000  108870.000000  Default  29.0000  29.0000
64             and   43234.000000   43234.000000  Default  28.0000  28.0000
36              of   48441.000000   48441.000000  Default  27.0000  27.0000
54              to   54717.000000   54717.000000  Default  26.0000  26.0000
...            ...            ...            ...      ...      ...      ...
8269  encyclopedia       1.532626      23.856547  Topic10  -7.1979   4.1670
302          world       1.764100    1083.555345  Topic10  -7.0572   0.4917
3901        master       1.502737     221.335192  Topic10  -7.2175   1.9197
2512       library       1.478815     281.847819  Topic10  -7.2336   1.6620
3357        cities       1.381747     138.829095  Topic10  -7.3015   2.3022

[656 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
118       1  0.726968  about
118       2  0.261442  about
118       3  0.011505  about
1737     10  0.796066   acne
1882      1  0.128191     ah
...     ...       ...    ...
117       2  0.259520   your
117       3  0.112369   your
6146      7  0.260494     zd
6146      8  0.734121     zd
6161      7  0.985289     zq

[1168 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 4, 2, 3, 9, 6, 8, 1, 5, 7])

In [20]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
tfidf = tfidf_vectorizer.fit_transform(documents)

In [21]:
nmf_model = NMF(n_components=10, random_state=42)
nmf_model.fit(tfidf)

NMF(n_components=10, random_state=42)

In [22]:
feature_names = tfidf_vectorizer.get_feature_names_out()

for topic_idx, topic in enumerate(nmf_model.components_):
    print(f"Topic {topic_idx}:")
    print([feature_names[i] for i in topic.argsort()[-10:]])

Topic 0:
['want', 'really', 'time', 'good', 've', 'know', 'think', 'like', 'don', 'just']
Topic 1:
['help', 'looking', 'anybody', 'info', 'hi', 'advance', 'mail', 'know', 'does', 'thanks']
Topic 2:
['say', 'church', 'christians', 'christian', 'christ', 'faith', 'believe', 'bible', 'jesus', 'god']
Topic 3:
['surrender', 'skepticism', 'intellect', 'shameful', 'pitt', 'cadre', 'n3jxp', 'chastity', 'dsl', 'geb']
Topic 4:
['nsa', 'algorithm', 'escrow', 'use', 'government', 'keys', 'clipper', 'encryption', 'chip', 'key']
Topic 5:
['bus', 'floppy', 'ide', 'controller', 'hard', 'card', 'disk', 'drives', 'scsi', 'drive']
Topic 6:
['email', '10', 'offer', 'price', 'space', 'com', 'sale', 'new', 'edu', '00']
Topic 7:
['version', 'problem', 'using', 'use', 'program', 'window', 'files', 'dos', 'file', 'windows']
Topic 8:
['league', 'win', 'hockey', 'play', 'season', 'players', 'games', 'year', 'team', 'game']
Topic 9:
['rights', 'law', 'gun', 'state', 'armenian', 'israeli', 'jews', 'government', 'i

In [23]:
nmf_topics = []
for topic in nmf_model.components_:
    top_words = [feature_names[i] for i in topic.argsort()[-10:]]
    nmf_topics.append(top_words)

coherence_nmf = CoherenceModel(
    topics=nmf_topics,
    texts=processed_docs,
    dictionary=dictionary,
    coherence='c_v'
)

print("NMF Coherence Score:", coherence_nmf.get_coherence())


NMF Coherence Score: 0.7367881584540329


In [27]:
topic_model = BERTopic()
topics, probs = topic_model.fit_transform(documents)

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [28]:
topic_model.get_topic_info()


,Topic,Count,Name,Representation,Representative_Docs
0,-1,3508,-1_the_to_and_of,"[the, to, and, of, is, for, in, you, it, that]",[In <lsjc8cINNmc1@saltillo.cs.utexas.edu> turp...
1,0,1067,0_team_game_he_season,"[team, game, he, season, games, players, play,...",[#21\tPETER AHOLA\t\tSeason: 2nd\nAcquired:\t'...
2,1,512,1_patients_msg_of_is,"[patients, msg, of, is, medical, in, it, healt...",[------------- cut here -----------------\n\n\...
3,2,363,2_key_chip_clipper_encryption,"[key, chip, clipper, encryption, keys, escrow,...",[The following document summarizes the Clipper...
4,3,307,3_ites_huh_art_each,"[ites, huh, art, each, why, of, , , , ]","[\n\t\t\t\t\tart\n, \nHuh?, ites:]"
...,...,...,...,...,...
135,134,10,134_ini_winini_files_autoexecbat,"[ini, winini, files, autoexecbat, windows, upd...","[\nNo. When the program is run, it loads 4 con..."
136,135,10,135_car_lights_lamps_backing,"[car, lights, lamps, backing, behind, reverse,...","[From article <1pq6i2$a1f@news.ysu.edu>, by ak..."
137,136,10,136_lines_airplanes_balls_pistrix,"[lines, airplanes, balls, pistrix, poles, tube...",[I got a question from my dad which I really c...
138,137,10,137_space_nasa_center_aerospace,"[space, nasa, center, aerospace, 00, publishes...",[I am looking for any information about the sp...


In [29]:
topic_model.visualize_topics()
